# Matt Manner, Connor Nickol, Josh Gen
## 3 Apr 23

Importing packages

In [1]:
import pandas as pd
import numpy as np
import os
#import torch

Mounting drives for importing from Google Drive


In [47]:
# Code for google colab
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

Importing data

In [57]:
players = pd.read_csv('players.csv')#'/content/drive/MyDrive/nfl-big-data-bowl-2023/players.csv')
plays = pd.read_csv('plays.csv')#'/content/drive/MyDrive/nfl-big-data-bowl-2023/plays.csv')
locations = pd.DataFrame()
for i in range(1,8):
    url = 'week'+str(i)+'.csv' #/content/drive/MyDrive/nfl-big-data-bowl-2023/week
    week_data = pd.read_csv(url)
    locations = pd.concat([locations, week_data])

Use GPU

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [52]:
games = pd.read_csv('games.csv')

In [53]:
games

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL
1,2021091200,2021,1,09/12/2021,13:00:00,ATL,PHI
2,2021091201,2021,1,09/12/2021,13:00:00,BUF,PIT
3,2021091202,2021,1,09/12/2021,13:00:00,CAR,NYJ
4,2021091203,2021,1,09/12/2021,13:00:00,CIN,MIN
...,...,...,...,...,...,...,...
117,2021103109,2021,8,10/31/2021,16:05:00,SEA,JAX
118,2021103110,2021,8,10/31/2021,16:25:00,DEN,WAS
119,2021103111,2021,8,10/31/2021,16:25:00,NO,TB
120,2021103112,2021,8,10/31/2021,20:20:00,MIN,DAL


In [54]:
games = games[['gameId','homeTeamAbbr']]

In [59]:
locations = locations.merge(games, on = 'gameId', how = 'left')

In [60]:
#

In [61]:
locations['uniqueplayId'] = locations['gameId'].astype(str) + locations['playId'].astype(str)

Need to make an adjacency matrix (nxnxt), edge feature matrix (exfxt), node feature matrix (nxfxt)

In [7]:
## Getting just one play for now and going to try the stack technique
test_adj = locations[['nflId','frameId','uniqueplayId','x','y']].set_index(['nflId','frameId'])
test_adj1 = test_adj[test_adj['uniqueplayId'] == '202109090097'].drop('uniqueplayId', axis = 1)

In [8]:
test_adj1

x      y
nflId   frameId              
25511.0 1        37.77  24.22
        2        37.78  24.22
        3        37.78  24.24
        4        37.73  24.25
        5        37.69  24.26
...                ...    ...
NaN     39       33.25  21.79
        40       36.55  19.21
        41       37.77  18.49
        42       38.91  17.79
        43       40.07  17.08

[989 rows x 2 columns]

In [ ]:
43*23

989

In [ ]:
## Get player labels for the example play
labels = locations[['nflId','frameId','uniqueplayId','x','y']]
labels = labels[labels['uniqueplayId'] == '202109090097']['nflId']
labels = np.unique(labels)

In [ ]:
## Looking at the first frame 
from scipy.spatial.distance import cdist
frame1 = pd.DataFrame(np.stack(test_adj1.groupby('frameId').apply(lambda x: cdist(x,x,'euclidean'))).astype(None)[0])
frame1 = frame1.set_index(labels)
frame1.columns = labels
frame1

,25511.0,35481.0,35634.0,39985.0,40151.0,41233.0,41263.0,42377.0,42403.0,42404.0,...,44904.0,44955.0,44962.0,46163.0,47996.0,52421.0,52459.0,53441.0,53504.0,NaN
25511.0,0.000000,5.938097,12.927444,1.973449,4.417080,9.080666,8.072329,4.627883,10.532260,4.421086,...,15.138246,5.773768,21.060888,3.981281,10.684779,4.427640,9.799765,6.384293,6.406942,5.153145
35481.0,5.938097,0.000000,7.590534,7.746206,5.295508,12.993325,10.763777,2.308354,4.608047,3.596804,...,9.282958,4.668212,17.614278,6.528859,9.660274,7.790314,13.105373,7.731093,3.600347,5.431179
35634.0,12.927444,7.590534,0.000000,14.885325,12.825327,20.575298,18.236516,9.865906,5.064000,11.121511,...,3.660833,11.862141,19.966477,14.116551,15.768339,15.380832,20.623312,15.106886,10.425685,12.845813
39985.0,1.973449,7.746206,14.885325,0.000000,4.945068,7.538289,7.080113,6.125112,12.284340,5.568492,...,17.009174,6.592367,21.661683,3.897602,10.833467,3.720497,8.449953,6.181691,7.567734,5.666816
40151.0,4.417080,5.295508,12.825327,4.945068,0.000000,7.781857,5.473874,2.987189,8.914084,1.705579,...,14.003517,1.741551,16.760251,1.540519,6.289777,2.805887,7.812324,2.590174,3.093558,0.743303
41233.0,9.080666,12.993325,20.575298,7.538289,7.781857,0.000000,3.003764,10.709883,16.623189,9.470190,...,21.733113,9.068302,19.983185,6.466970,9.165948,5.241040,1.492146,5.938527,10.613802,7.902569
41263.0,8.072329,10.763777,18.236516,7.080113,5.473874,3.003764,0.000000,8.457435,13.994892,7.166987,...,19.098482,6.442802,17.253420,4.523549,6.236986,3.657280,2.435262,3.180393,7.970056,5.391030
42377.0,4.627883,2.308354,9.865906,6.125112,2.987189,10.709883,8.457435,0.000000,6.307963,1.293406,...,11.282491,2.597865,17.055676,4.261983,7.943815,5.544448,10.797541,5.455795,2.151279,3.162183
42403.0,10.532260,4.608047,5.064000,12.284340,8.914084,16.623189,13.994892,6.307963,0.000000,7.331255,...,5.111653,7.570740,15.524642,10.393060,10.796342,11.690308,16.428551,10.814735,6.029594,8.720929
42404.0,4.421086,3.596804,11.121511,5.568492,1.705579,9.470190,7.166987,1.293406,7.331255,0.000000,...,12.385350,1.606487,16.733541,3.076004,7.019095,4.373786,9.517069,4.169652,2.011890,1.888200


In [ ]:
## Performing some check to make sure the ids are lined up correctly 
test_play1 = locations[['nflId','frameId','uniqueplayId','x','y']]
test_play1 = test_play1[test_play1['uniqueplayId'] == '202109090097']
test_play1 = test_play1[test_play1['frameId'] == 1]
test_play1

,nflId,frameId,uniqueplayId,x,y
0,25511.0,1,202109090097,37.77,24.22
43,35481.0,1,202109090097,41.08,29.15
86,35634.0,1,202109090097,40.99,36.74
129,39985.0,1,202109090097,37.54,22.26
172,40151.0,1,202109090097,42.18,23.97
215,41233.0,1,202109090097,42.01,16.19
258,41263.0,1,202109090097,43.66,18.70
301,42377.0,1,202109090097,41.55,26.89
344,42403.0,1,202109090097,44.03,32.69
387,42404.0,1,202109090097,41.95,25.66


In [ ]:
import math
p = [37.77, 24.22] 
q = [43.69, 26.67] 

# Calculate Euclidean distance
print (math.dist(p, q))

6.406941548039903


In [ ]:
## Attempting to get the adjacency matrices for all plays
from scipy.spatial.distance import cdist
all_plays  = locations[['nflId','frameId','uniqueplayId','x','y']].set_index(['nflId','frameId','uniqueplayId'])
adj_matrices = all_plays.groupby(['uniqueplayId','frameId']).apply(lambda x: cdist(x,x,'euclidean'))

In [ ]:
adj_matrices.shape

(318923,)

We have 318,923 frames. For each of these frames there is a 23x23 distance matrix.

In [ ]:
adj_matrices

uniqueplayId    frameId
20210909001078  1          [[0.0, 3.3509550877324523, 2.3168297304722225,...
                2          [[0.0, 3.320963715550051, 2.2939267643061307, ...
                3          [[0.0, 3.2909725006447585, 2.2770595073471402,...
                4          [[0.0, 3.2609814473559964, 2.265965577849759, ...
                5          [[0.0, 3.2403857795021893, 2.2717834403833477,...
                                                 ...                        
2021102500933   59         [[0.0, 15.854731155084279, 1.6911830178901415,...
                60         [[0.0, 15.856437178635062, 1.944453650771859, ...
                61         [[0.0, 15.882833500355027, 2.242074931843273, ...
                62         [[0.0, 15.919447226584214, 2.536454217998032, ...
                63         [[0.0, 15.981267158770601, 2.8402288640178264,...
Length: 318923, dtype: object

In [ ]:
binary_adj = adj_matrices
for frame in range(len(binary_adj)):
  binary_adj[frame] = (binary_adj[frame] < 10).astype(int)

In [ ]:
binary_adj

uniqueplayId    frameId
20210909001078  1          [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,...
                2          [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,...
                3          [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,...
                4          [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,...
                5          [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,...
                                                 ...                        
2021102500933   59         [[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,...
                60         [[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,...
                61         [[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,...
                62         [[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,...
                63         [[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,...
Length: 318923, dtype: object

## For the edge matrix, we need to figure out how to make it number of edges by number of features keeping track of who is connected to who

## Working on the feature matrix. Things to implement:
- Cut the play after 3? seconds
- Corner orientation or maybe all defensive player orientation
- Yard line, down and distance, score margin, quarter all possible additions
- Personel package on the field? 

In [62]:
## Standardizing the ball to always be the same spot

## Filter out some of the coverages
plays = plays[plays.pff_passCoverage != 'Bracket']
plays = plays[plays.pff_passCoverage != 'Goal Line']
plays = plays[plays.pff_passCoverage != 'Miscellaneous']
plays = plays[plays.pff_passCoverage != 'Prevent']
plays = plays[plays.pff_passCoverage != 'Red Zone']

plays['uniqueplayId'] = plays['gameId'].astype(str) + plays['playId'].astype(str)

# merge the locational dataset with the plays dataset to remove these coverages in locational dataset
locations = plays.merge(locations, on='uniqueplayId')

In [63]:
# Dropping all of the rows that occur before the snap of the ball
snap_time = locations.loc[locations['event'] == 'ball_snap',['uniqueplayId', 'frameId']].rename(columns = {'frameId':'ball_snap_frame'})
snap_time_unique = snap_time.drop_duplicates(subset = 'uniqueplayId')

In [64]:
locations = locations.merge(snap_time_unique, on = 'uniqueplayId', how = 'left')
locations

,gameId_x,playId_x,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,x,y,s,a,dis,o,dir,event,homeTeamAbbr,ball_snap_frame
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,TB,6.0
1,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,TB,6.0
2,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,TB,6.0
3,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,TB,6.0
4,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,TB,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891830,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.84,18.54,4.19,3.60,0.44,NaN,NaN,None,SEA,7.0
6891831,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.74,18.24,3.39,4.45,0.32,NaN,NaN,None,SEA,7.0
6891832,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.66,17.97,2.74,4.55,0.27,NaN,NaN,None,SEA,7.0
6891833,2021102500,3945,"(1:23) (No Huddle, Shotgun) G.Smith sacked at ...",4,3,18,SEA,NO,SEA,17,...,103.59,17.74,2.22,4.87,0.24,NaN,NaN,None,SEA,7.0


In [65]:
locations = locations[~(locations['frameId'] < locations['ball_snap_frame'])]

In [66]:
## Dropping any frames after 3 seconds 
locations['end_frame'] = locations['ball_snap_frame'] + 30

<ipython-input-66-cfefbb3e0bfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['end_frame'] = locations['ball_snap_frame'] + 30


In [67]:
locations = locations[~(locations['frameId'] > locations['end_frame'])]

In [68]:
locations.columns

Index(['gameId_x', 'playId_x', 'playDescription', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'passResult', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'foulName1', 'foulNFLId1',
       'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
       'absoluteYardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInBox', 'personnelD', 'dropBackType', 'pff_playAction',
       'pff_passCoverage', 'pff_passCoverageType', 'uniqueplayId', 'gameId_y',
       'playId_y', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event',
       'homeTeamAbbr', 'ball_snap_frame', 'end_frame'],
      dtype='object')

In [69]:
## Dropping columns not using for analysis
locations = locations[['gameId_x', 'playId_x', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore','homeTeamAbbr',
       'absoluteYardlineNumber', 'pff_passCoverage', 'uniqueplayId', 'nflId', 'frameId', 'time', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'o', 'event',]]

In [70]:
locations

,gameId_x,playId_x,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,gameClock,preSnapHomeScore,...,frameId,time,team,playDirection,x,y,s,a,o,event
5,2021090900,97,1,3,2,TB,DAL,33,13:33,0,...,6,2021-09-10 00:26:31,TB,right,37.64,24.26,0.35,0.53,144.42,ball_snap
6,2021090900,97,1,3,2,TB,DAL,33,13:33,0,...,7,2021-09-10 00:26:31,TB,right,37.56,24.26,0.54,1.05,137.49,None
7,2021090900,97,1,3,2,TB,DAL,33,13:33,0,...,8,2021-09-10 00:26:31,TB,right,37.47,24.25,0.80,1.85,131.95,None
8,2021090900,97,1,3,2,TB,DAL,33,13:33,0,...,9,2021-09-10 00:26:31,TB,right,37.38,24.24,0.99,2.03,129.85,None
9,2021090900,97,1,3,2,TB,DAL,33,13:33,0,...,10,2021-09-10 00:26:32,TB,right,37.27,24.23,1.19,1.82,123.79,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891804,2021102500,3945,4,3,18,SEA,NO,17,01:23,10,...,33,2021-10-26 03:14:36,football,left,102.23,27.11,1.48,0.57,NaN,None
6891805,2021102500,3945,4,3,18,SEA,NO,17,01:23,10,...,34,2021-10-26 03:14:36,football,left,102.35,27.20,1.56,0.81,NaN,None
6891806,2021102500,3945,4,3,18,SEA,NO,17,01:23,10,...,35,2021-10-26 03:14:36,football,left,102.49,27.27,1.61,0.94,NaN,None
6891807,2021102500,3945,4,3,18,SEA,NO,17,01:23,10,...,36,2021-10-26 03:14:36,football,left,102.64,27.33,1.64,1.55,NaN,None


In [71]:
## Dropping any rows after the ball has been thrown
locations['event'].value_counts()

None                         4445026
ball_snap                     162633
pass_forward                   93564
autoevent_passforward          46299
play_action                    37973
autoevent_ballsnap             29647
pass_arrived                    3243
autoevent_passinterrupted       1656
pass_tipped                     1541
run                             1173
first_contact                    644
qb_sack                          575
shift                            575
qb_strip_sack                    460
man_in_motion                    253
pass_outcome_incomplete          207
pass_outcome_caught              207
fumble                           207
handoff                          207
fumble_offense_recovered         161
line_set                         138
lateral                           23
huddle_break_offense              23
Name: event, dtype: int64

In [72]:
end_time = locations.loc[locations['event'].isin(['pass_forward','autoevent_passforward','qb_sack','qb_strip_sack','handoff'])]
end_time = end_time.groupby('uniqueplayId').agg('min').reset_index()

In [73]:
end_time['end_frame'] = end_time['frameId']
end_time = end_time[['uniqueplayId','end_frame']]

In [74]:
locations = locations.merge(end_time, on = 'uniqueplayId', how = 'left')

In [75]:
locations['end_frame'] = locations['end_frame'].fillna(100)

In [76]:
locations = locations[locations['frameId'] < locations['end_frame']]

In [77]:
locations = locations.drop(['gameId_x','playId_x','end_frame'],axis = 1)

In [78]:
locations

,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,homeTeamAbbr,...,frameId,time,team,playDirection,x,y,s,a,o,event
0,1,3,2,TB,DAL,33,13:33,0,0,TB,...,6,2021-09-10 00:26:31,TB,right,37.64,24.26,0.35,0.53,144.42,ball_snap
1,1,3,2,TB,DAL,33,13:33,0,0,TB,...,7,2021-09-10 00:26:31,TB,right,37.56,24.26,0.54,1.05,137.49,None
2,1,3,2,TB,DAL,33,13:33,0,0,TB,...,8,2021-09-10 00:26:31,TB,right,37.47,24.25,0.80,1.85,131.95,None
3,1,3,2,TB,DAL,33,13:33,0,0,TB,...,9,2021-09-10 00:26:31,TB,right,37.38,24.24,0.99,2.03,129.85,None
4,1,3,2,TB,DAL,33,13:33,0,0,TB,...,10,2021-09-10 00:26:32,TB,right,37.27,24.23,1.19,1.82,123.79,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826430,4,3,18,SEA,NO,17,01:23,10,13,SEA,...,33,2021-10-26 03:14:36,football,left,102.23,27.11,1.48,0.57,NaN,None
4826431,4,3,18,SEA,NO,17,01:23,10,13,SEA,...,34,2021-10-26 03:14:36,football,left,102.35,27.20,1.56,0.81,NaN,None
4826432,4,3,18,SEA,NO,17,01:23,10,13,SEA,...,35,2021-10-26 03:14:36,football,left,102.49,27.27,1.61,0.94,NaN,None
4826433,4,3,18,SEA,NO,17,01:23,10,13,SEA,...,36,2021-10-26 03:14:36,football,left,102.64,27.33,1.64,1.55,NaN,None


In [79]:
football = locations.loc[locations['team'] == 'football',
                               ['uniqueplayId', 'x', 'y','event']
                               ].rename(columns={'x':'football_x',
                                                 'y':'football_y'})
locations = football[football['event'] == 'ball_snap'].merge(locations, on='uniqueplayId', how='right')

In [80]:
locations

,uniqueplayId,football_x,football_y,event_x,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,...,frameId,time,team,playDirection,x,y,s,a,o,event_y
0,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,6,2021-09-10 00:26:31,TB,right,37.64,24.26,0.35,0.53,144.42,ball_snap
1,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,7,2021-09-10 00:26:31,TB,right,37.56,24.26,0.54,1.05,137.49,None
2,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,8,2021-09-10 00:26:31,TB,right,37.47,24.25,0.80,1.85,131.95,None
3,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,9,2021-09-10 00:26:31,TB,right,37.38,24.24,0.99,2.03,129.85,None
4,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,10,2021-09-10 00:26:32,TB,right,37.27,24.23,1.19,1.82,123.79,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4340647,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,33,2021-10-26 03:14:36,football,left,102.23,27.11,1.48,0.57,NaN,None
4340648,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,34,2021-10-26 03:14:36,football,left,102.35,27.20,1.56,0.81,NaN,None
4340649,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,35,2021-10-26 03:14:36,football,left,102.49,27.27,1.61,0.94,NaN,None
4340650,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,36,2021-10-26 03:14:36,football,left,102.64,27.33,1.64,1.55,NaN,None


In [81]:
locations['new_x'] = abs(locations['x']-locations['football_x'])
locations['new_y'] = np.where(locations['playDirection']=='right', 
                                   locations['football_y']-locations['y'],
                                   locations['y']-locations['football_y'])

In [82]:
locations['Defense'] = np.where(locations['team']== locations['defensiveTeam'], 1, 0)

In [83]:
flipped_o = locations.loc[locations['playDirection'] == 'right',['uniqueplayId','nflId','frameId', 'o']]
flipped_o['o'] = np.where(flipped_o['o'] > 180,
                         flipped_o['o'] - 180,
                         360 - flipped_o['o'])

In [84]:
flipped_o

,uniqueplayId,nflId,frameId,o
0,202109090097,25511.0,6,215.58
1,202109090097,25511.0,7,222.51
2,202109090097,25511.0,8,228.05
3,202109090097,25511.0,9,230.15
4,202109090097,25511.0,10,236.21
...,...,...,...,...
4338646,20211025003735,NaN,33,NaN
4338647,20211025003735,NaN,34,NaN
4338648,20211025003735,NaN,35,NaN
4338649,20211025003735,NaN,36,NaN


In [85]:
locations.update(flipped_o)

In [89]:
locations.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'event_x', 'quarter',
       'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'homeTeamAbbr', 'absoluteYardlineNumber',
       'pff_passCoverage', 'nflId', 'frameId', 'time', 'team', 'playDirection',
       'x', 'y', 's', 'a', 'o', 'event_y', 'new_x', 'new_y', 'Defense'],
      dtype='object')

In [90]:
locations['score_d'] = locations['preSnapHomeScore'] - locations['preSnapVisitorScore']

In [97]:
locations['score_d'] = np.where(locations['defensiveTeam'] == locations['homeTeamAbbr'], locations['score_d'] * 1, locations['score_d'] * -1)

In [98]:
locations

,uniqueplayId,football_x,football_y,event_x,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,...,x,y,s,a,o,event_y,new_x,new_y,Defense,score_d
0,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,37.64,24.26,0.35,0.53,215.58,ball_snap,3.92,-0.34,0,0
1,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,37.56,24.26,0.54,1.05,222.51,None,4.00,-0.34,0,0
2,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,37.47,24.25,0.80,1.85,228.05,None,4.09,-0.33,0,0
3,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,37.38,24.24,0.99,2.03,230.15,None,4.18,-0.32,0,0
4,202109090097,41.56,23.92,ball_snap,1,3,2,TB,DAL,33,...,37.27,24.23,1.19,1.82,236.21,None,4.29,-0.31,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4340647,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,102.23,27.11,1.48,0.57,NaN,None,7.74,0.60,0,3
4340648,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,102.35,27.20,1.56,0.81,NaN,None,7.86,0.69,0,3
4340649,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,102.49,27.27,1.61,0.94,NaN,None,8.00,0.76,0,3
4340650,20211025003945,94.49,26.51,ball_snap,4,3,18,SEA,NO,17,...,102.64,27.33,1.64,1.55,NaN,None,8.15,0.82,0,3


In [144]:
## Attempting to get the adjacency matrices for all plays
from scipy.spatial.distance import cdist
all_plays  = locations[['nflId','frameId','uniqueplayId','new_x','new_y']].set_index(['nflId','frameId','uniqueplayId'])
dis_matrices = all_plays.groupby(['uniqueplayId','frameId']).apply(lambda x: cdist(x,x,'euclidean'))

In [150]:
dis_matrices[0]

array([[ 0.        ,  3.22013975,  1.2980755 , 11.26812318,  5.88153891,
         7.12511754, 14.75107115,  2.4060133 ,  4.96424214,  3.29611893,
         0.96208108, 23.69477791, 23.32021441,  5.98965775,  4.3363118 ,
         3.43960754, 15.39321929, 12.584705  ,  5.45641824, 13.59609135,
        17.69420809,  2.42917682,  2.17165835],
       [ 3.22013975,  0.        ,  2.07125566, 14.3285903 ,  3.34400957,
        10.34278492, 11.545432  ,  5.6217435 ,  8.17655184,  5.82722061,
         3.67947007, 26.71495836, 26.54031839,  2.78469029,  6.7416096 ,
         6.6197432 , 12.58644112, 11.84120349,  8.4395379 , 16.81097261,
        19.9321775 ,  5.52268051,  5.17820432],
       [ 1.2980755 ,  2.07125566,  0.        , 12.56592217,  5.11786088,
         8.36280455, 13.51469201,  3.66629241,  6.15003252,  4.45558077,
         1.60853971, 24.99284898, 24.5327903 ,  4.77177116,  5.47483333,
         4.57052513, 14.55709449, 12.67738932,  6.74866653, 14.77915086,
        18.85167367,  3.4556

In [146]:
binary_adj = dis_matrices.copy()
for frame in range(len(binary_adj)):
  binary_adj[frame] = (binary_adj[frame] < 10).astype(int)

In [147]:
binary_adj[0]

array([[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1],
       [1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1],
       [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1],
       [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        0],
       [1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        1],
       [1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
        1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0],
       [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1],
       [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
        1],
       [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1],
       [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0

In [148]:
edge_features = dis_matrices.copy()
for frame in range(len(edge_features)):
    edge_features[frame] = np.array([-13*(xi) + 131.0 for xi in edge_features[frame]])

In [149]:
edge_features[0]

array([[ 1.31000000e+02,  8.91381833e+01,  1.14125019e+02,
        -1.54856013e+01,  5.45399941e+01,  3.83734719e+01,
        -6.07639249e+01,  9.97218271e+01,  6.64648522e+01,
         8.81504539e+01,  1.18492946e+02, -1.77032113e+02,
        -1.72162787e+02,  5.31344492e+01,  7.46279466e+01,
         8.62851020e+01, -6.91118507e+01, -3.26011650e+01,
         6.00665629e+01, -4.57491876e+01, -9.90247052e+01,
         9.94207014e+01,  1.02768441e+02],
       [ 8.91381833e+01,  1.31000000e+02,  1.04073676e+02,
        -5.52716739e+01,  8.75278756e+01, -3.45620402e+00,
        -1.90906160e+01,  5.79173345e+01,  2.47048261e+01,
         5.52461321e+01,  8.31668891e+01, -2.16294459e+02,
        -2.14024139e+02,  9.47990263e+01,  4.33590752e+01,
         4.49433384e+01, -3.26237345e+01, -2.29356453e+01,
         2.12860073e+01, -8.75426439e+01, -1.28118308e+02,
         5.92051534e+01,  6.36833438e+01],
       [ 1.14125019e+02,  1.04073676e+02,  1.31000000e+02,
        -3.23569882e+01,  6.4

In [151]:
locations.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'event_x', 'quarter',
       'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'homeTeamAbbr', 'absoluteYardlineNumber',
       'pff_passCoverage', 'nflId', 'frameId', 'time', 'team', 'playDirection',
       'x', 'y', 's', 'a', 'o', 'event_y', 'new_x', 'new_y', 'Defense',
       'score_d'],
      dtype='object')

In [168]:
node_features = locations[['uniqueplayId', 'quarter',
       'down', 'yardsToGo', 'defensiveTeam',
       'yardlineNumber', 'absoluteYardlineNumber','nflId', 
        'frameId', 's', 'o', 'new_x', 'new_y', 'Defense',
       'score_d']].set_index(['uniqueplayId','frameId','nflId'])

In [169]:
## Where does our response go? In the node feature matrix or separate?

In [170]:
node_features[:50]

quarter  down  yardsToGo defensiveTeam  \
uniqueplayId frameId nflId                                             
202109090097 6.0     25511.0        1     3          2           DAL   
             7.0     25511.0        1     3          2           DAL   
             8.0     25511.0        1     3          2           DAL   
             9.0     25511.0        1     3          2           DAL   
             10.0    25511.0        1     3          2           DAL   
             11.0    25511.0        1     3          2           DAL   
             12.0    25511.0        1     3          2           DAL   
             13.0    25511.0        1     3          2           DAL   
             14.0    25511.0        1     3          2           DAL   
             15.0    25511.0        1     3          2           DAL   
             16.0    25511.0        1     3          2           DAL   
             17.0    25511.0        1     3          2           DAL   
             18.0    25511.0        1     3          2           DAL   
             19.0    25511.0        1     3          2           DAL   
             20.0    25511.0        1     3          2           DAL   
             21.0    25511.0        1     3          2           DAL   
             22.0    25511.0        1     3          2           DAL   
             23.0    25511.0        1     3          2           DAL   
             24.0    25511.0        1     3          2           DAL   
             25.0    25511.0        1     3          2           DAL   
             26.0    25511.0        1     3          2           DAL   
             27.0    25511.0        1     3          2           DAL   
             28.0    25511.0        1     3          2           DAL   
             29.0    25511.0        1     3          2           DAL   
             30.0    25511.0        1     3          2           DAL   
             31.0    25511.0        1     3          2           DAL   
             32.0    25511.0        1     3          2           DAL   
             33.0    25511.0        1     3          2           DAL   
             34.0    25511.0        1     3          2           DAL   
             35.0    25511.0        1     3          2           DAL   
             36.0    25511.0        1     3          2           DAL   
             6.0     35481.0        1     3          2           DAL   
             7.0     35481.0        1     3          2           DAL   
             8.0     35481.0        1     3          2           DAL   
             9.0     35481.0        1     3          2           DAL   
             10.0    35481.0        1     3          2           DAL   
             11.0    35481.0        1     3          2           DAL   
             12.0    35481.0        1     3          2           DAL   
             13.0    35481.0        1     3          2           DAL   
             14.0    35481.0        1     3          2           DAL   
             15.0    35481.0        1     3          2           DAL   
             16.0    35481.0        1     3          2           DAL   
             17.0    35481.0        1     3          2           DAL   
             18.0    35481.0        1     3          2           DAL   
             19.0    35481.0        1     3          2           DAL   
             20.0    35481.0        1     3          2           DAL   
             21.0    35481.0        1     3          2           DAL   
             22.0    35481.0        1     3          2           DAL   
             23.0    35481.0        1     3          2           DAL   
             24.0    35481.0        1     3          2           DAL   

                              yardlineNumber  absoluteYardlineNumber     s  \
uniqueplayId frameId nflId                                                   
202109090097 6.0     25511.0              33                    43.0  0.35   
             7.0     25511.0              33       

In [158]:
y = locations[['uniqueplayId','frameId','nflId','pff_passCoverage']].set_index(['uniqueplayId','frameId','nflId'])

In [171]:
binary_adj.to_pickle('adj_mat.pkl')

In [172]:
edge_features.to_pickle('edge_features.pkl')

In [173]:
node_features.to_pickle('node_features.pkl')

In [174]:
y.to_pickle('y.pkl')